# Recommender system implementation 

In [3]:
# Import libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json
from bertopic import BERTopic
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
import numpy as np

# Path to my json file with the openalex results (1000 papers)
json_path = "/Users/sanduandrei/Desktop/Luiss 1st Year/Data Science in Action/Project/openalex_results_clean.json"

# Open and load the JSON data
with open(json_path, 'r') as f:
    data = json.load(f)

# Convert to DataFrame 
df = pd.DataFrame(data)
df_clean = df[df['abstract'].notna()].copy()
docs = (df_clean['title'] + ". " + df_clean['abstract']).tolist()

# Topic modeling
topic_model = BERTopic.load("MaartenGr/BERTopic_Wikipedia")
topics, probs = topic_model.transform(docs)

df_clean['topic_id'] = topics
df_clean['topic_label'] = df_clean['topic_id'].apply(
    lambda x: topic_model.topic_labels_[x] if x != -1 and x < len(topic_model.topic_labels_) else "Unknown"
)

# Embedding
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = embedding_model.encode(docs, show_progress_bar=True)
df_clean['embedding'] = list(embeddings)

# Finalize main paper DataFrame
papers_df = df_clean.copy()
paper_embeddings = np.vstack(papers_df['embedding'].values)


RuntimeError: CPU dispatcher tracer already initlized

Maybe differeniate between title and abstract in docs ... 

In [21]:
display(df_clean)

,id,doi,title,display_name,relevance_score,publication_year,publication_date,ids,language,primary_location,...,referenced_works,related_works,cited_by_api_url,counts_by_year,updated_date,created_date,abstract,topic_id,topic_label,embedding
0,https://openalex.org/W3047327247,https://doi.org/10.1080/13675567.2020.1803246,Machine learning demand forecasting and supply...,Machine learning demand forecasting and supply...,851.878300,2020,2020-08-04,{'openalex': 'https://openalex.org/W3047327247...,en,"{'is_oa': True, 'landing_page_url': 'https://d...",...,"[https://openalex.org/W122374401, https://open...","[https://openalex.org/W4313057686, https://ope...",https://api.openalex.org/works?filter=cites:W3...,"[{'year': 2025, 'cited_by_count': 13}, {'year'...",2025-03-19T09:40:40.806765,2020-08-10,"In many supply chains, firms staged in upstrea...",181,181_neural_neuron_neurons_convolutions,"[-0.063067734, -0.04766745, 0.06032231, 0.0451..."
1,https://openalex.org/W3024362711,https://doi.org/10.3390/su12104035,Influences of the Industry 4.0 Revolution on t...,Influences of the Industry 4.0 Revolution on t...,754.795500,2020,2020-05-14,{'openalex': 'https://openalex.org/W3024362711...,en,"{'is_oa': True, 'landing_page_url': 'https://d...",...,"[https://openalex.org/W1560042744, https://ope...","[https://openalex.org/W648638662, https://open...",https://api.openalex.org/works?filter=cites:W3...,"[{'year': 2025, 'cited_by_count': 13}, {'year'...",2025-03-27T13:03:33.223437,2020-05-21,"Automation and digitalization, as long-term ev...",1821,1821_commerce_retailers_shopping_retailing,"[0.005194977, -0.036903504, 0.006172834, 0.001..."
2,https://openalex.org/W2923129012,https://doi.org/10.1155/2019/9067367,An Improved Demand Forecasting Model Using Dee...,An Improved Demand Forecasting Model Using Dee...,726.105000,2019,2019-01-01,{'openalex': 'https://openalex.org/W2923129012...,en,"{'is_oa': True, 'landing_page_url': 'https://d...",...,"[https://openalex.org/W124243540, https://open...","[https://openalex.org/W4375867731, https://ope...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2025, 'cited_by_count': 4}, {'year':...",2025-03-23T10:02:26.057098,2019-04-01,Demand forecasting is one of the main issues o...,181,181_neural_neuron_neurons_convolutions,"[-0.13550715, -0.029657131, 0.015677836, 0.020..."
3,https://openalex.org/W2980994438,https://doi.org/10.1016/j.ijforecast.2019.07.001,DeepAR: Probabilistic forecasting with autoreg...,DeepAR: Probabilistic forecasting with autoreg...,693.075000,2019,2019-10-19,{'openalex': 'https://openalex.org/W2980994438...,en,"{'is_oa': True, 'landing_page_url': 'https://d...",...,"[https://openalex.org/W129305155, https://open...","[https://openalex.org/W3188413760, https://ope...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2025, 'cited_by_count': 57}, {'year'...",2025-03-27T11:10:07.481385,2019-10-25,"Probabilistic forecasting, i.e., estimating a ...",181,181_neural_neuron_neurons_convolutions,"[-0.15686408, -0.0609649, 0.014155921, 0.01204..."
4,https://openalex.org/W4387379065,https://doi.org/10.1080/08874417.2023.2261010,The Potential of Generative Artificial Intelli...,The Potential of Generative Artificial Intelli...,688.953300,2023,2023-10-05,{'openalex': 'https://openalex.org/W4387379065...,en,"{'is_oa': False, 'landing_page_url': 'https://...",...,"[https://openalex.org/W2772633599, https://ope...","[https://openalex.org/W4380551139, https://ope...",https://api.openalex.org/works?filter=cites:W4...,"[{'year': 2025, 'cited_by_count': 46}, {'year'...",2025-03-23T10:47:46.228229,2023-10-06,ABSTRACTIn a short span of time since its intr...,181,181_neural_neuron_neurons_convolutions,"[-0.09569368, -0.016893514, 0.06916384, 0.0253..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,https://openalex.org/W2798697510,https://doi.org/10.1145/3209978.3210211,Machine Learning @ Amazon,Machine Learning @ Amazon,82.775730,2018,2018

In [23]:
# print(docs)

In [ ]:
def recommend_similar_papers_from_query(query_text, top_k=5, boost_topic=True):
    # Get query embedding 
    query_embedding = embedding_model.encode(query_text, convert_to_tensor=True)

    # Get the cosine scores 
    cosine_scores = util.pytorch_cos_sim(query_embedding, paper_embeddings)[0].cpu().numpy()

    # Get the predicted topic for the query
    if boost_topic:
        query_topic, _ = topic_model.transform([query_text])
        topic_boost_mask = (papers_df['topic_id'] == query_topic[0]).values.astype(float)
        cosine_scores += 0.05 * topic_boost_mask  # Boost same-topic papers slightly
    
    # Get the top results 
    top_results = np.argsort(-cosine_scores)[:top_k]

    # And print them 
    for idx in top_results:
        print(f"Title: {papers_df.iloc[idx]['title']}")
        print(f"Score: {cosine_scores[idx]:.4f}")
        print(f"Topic: {papers_df.iloc[idx]['topic_label']}")
        print(f"Abstract: {papers_df.iloc[idx]['abstract']}\n")




In [ ]:
# Define your query
QUERY = "Reinforcement learning for automated pricing in e-commerce"

# Get recommendations
recommend_similar_papers_from_query(query_text=query, top_k=5)





Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-04-02 18:22:20,496 - BERTopic - Predicting topic assignments through cosine similarity of topic and document embeddings.


Title: Dynamic Pricing Model of E-Commerce Platforms Based on Deep Reinforcement Learning
Score: 0.7884
Topic: 1821_commerce_retailers_shopping_retailing
Abstract: With the continuous development of artificial intelligence technology, its application field has gradually expanded. To further apply the deep reinforcement learning technology to the field of dynamic pricing, we build an intelligent dynamic pricing system, introduce the reinforcement learning technology related to dynamic pricing, and introduce existing research on the number of suppliers (single supplier and multiple suppliers), environmental models, and selection algorithms. A two-period dynamic pricing game model is designed to assess the optimal pricing strategy for e-commerce platforms under two market conditions and two consumer participation conditions. The first step is to analyze the pricing strategies of e-commerce platforms in mature markets, analyze the optimal pricing and profits of various enterprises under di

In [ ]:
QUERY = "Effectiveness of personalized promotion strategies using customer data"
recommend_similar_papers_from_query(query_text=query, top_k=5)




Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-04-02 18:22:26,148 - BERTopic - Predicting topic assignments through cosine similarity of topic and document embeddings.


Title: Hyper-Personalization
Score: 0.6606
Topic: 1250_marketing_advertising_market_consumers
Abstract: Personalization is widely used to attract and retain customers in online business addressing one size fits all issues, but little is addressed to contextualise users' real-time needs. E-commerce website owners use these strategies for customer-centric marketing through enhanced experience but fail in designing effective personalization due to the dynamic nature of users' needs and pace of information exposure. To address this, this chapter explores hyper-personalization strategies to overcome users' implicit need to be served better. The research presents a hyper-personalization process with learning (ML) and artificial intelligence (AI) techniques for marketing functions like segmentation, targeting, and positioning based on real-time analytics throughout the customer journey and key factors driving effective customer-centric marketing. This chapter facilitates marketers to use AI-e

In [ ]:
QUERY = "Natural language processing for sentiment-driven pricing strategies"
recommend_similar_papers_from_query(query_text=query, top_k=5)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-04-02 18:22:34,215 - BERTopic - Predicting topic assignments through cosine similarity of topic and document embeddings.


Title: Research on Chinese Consumers’ Attitudes Analysis of Big-Data Driven Price Discrimination Based on Machine Learning
Score: 0.6003
Topic: 1250_marketing_advertising_market_consumers
Abstract: From the end of 2018 in China, the Big-data Driven Price Discrimination (BDPD) of online consumption raised public debate on social media. To study the consumers' attitude about the BDPD, this study constructed a semantic recognition frame to deconstruct the Affection-Behavior-Cognition (ABC) consumer attitude theory using machine learning models inclusive of the Labeled Latent Dirichlet Allocation (LDA), Long Short-Term Memory (LSTM), and Snow Natural Language Processing (NLP), based on social media comments text dataset. Similar to the questionnaires published results, this article verified that 61% of consumers expressed negative sentiment toward BDPD in general. Differently, on a finer scale, this study further measured the negative sentiments that differ significantly among different to